## Classy classification

Нашу задачу можно свести к задаче классификации предложений текста между двумя классами - целевым и nontype


Classy classification - библиотека, в которой содержатся модели для классификации текстов. 

### Установка

In [ ]:
!pip install classy-classification
!pip install pymorphy2
# !wget https://github.com/buriy/spacy-ru/releases/download/v2.3_beta/ru2_combined_400ks_96.zip
# !unzip ru2_combined_400ks_96.zip
# !pip install spacy==2.1.9
# !git clone -b v2.1 https://github.com/buriy/spacy-ru.git && cp -r ./spacy-ru/ru2/. /content/ru2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pystemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download ru_core_news_lg

2023-04-15 15:23:12.702457: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 15:23:14.291113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 2.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')


### Импорты

In [ ]:
from utils import accuracy_score, choose_minimum_rate_sentence
from sklearn.model_selection import train_test_split
from random import randint
from pprint import pprint

import matplotlib.pyplot as plt
import classy_classification
import seaborn as sns
import pandas as pd
import numpy as np
import pickle
import spacy


In [ ]:

with open('/content/contract_enforcement.pkl', 'rb') as fp:
  contract_enforcement = pickle.load(fp)
print(len(contract_enforcement))
contract_enforcement = pd.Series(contract_enforcement)

contract_enforcement = contract_enforcement[~contract_enforcement.isna()]
contract_enforcement = contract_enforcement.values
len(contract_enforcement)

988


984

In [ ]:
with open('/content/garantee_enforcement.pkl', 'rb') as fp:
  garantee_enforcement = pickle.load(fp)
garantee_enforcement = pd.Series(garantee_enforcement)
print(len(garantee_enforcement))

garantee_enforcement = garantee_enforcement[~garantee_enforcement.isna()]
garantee_enforcement = garantee_enforcement.values
len(garantee_enforcement)


811


508

In [ ]:
with open('/content/nonetype.pkl', 'rb') as fp:
  nontype = pickle.load(fp)


11232

In [ ]:
nonetype[:5]

array(['Версия с 04.07.2022 года',
       'Ю «Государственное автономное учреждение',
       "Саха (Якутия) ''Республиканская больница №1 -",
       "Национальный центр медицины''»",
       'Подписано усиленной квалифицированной электронной'], dtype='<U867')

In [ ]:
df = pd.read_json('/content/train_with_splitted_texts_extended.json', encoding='utf-8')

In [ ]:
df.head()

,id,text,label,target,not_found
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...,NaN
1,854885310,"[ТРЕБОВАНИЯ, К СОДЕРЖАНИЮ ЗАЯВКИ участника зап...",обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...,NaN
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%,NaN


### построение модели ru

попробуем построить модель для "обеспечение исполнения контракта"

In [ ]:
# будем использовать батчи по 100 значений для более быстрой работы
contract_enforcement_100 = np.random.choice(contract_enforcement, 100, replace=False)
nonetype_100 = np.random.choice(nonetype, 100, replace=False)



data={
    'обеспечение исполнения контракта': list(contract_enforcement_100),
    'nonetype': list(nonetype_100)}
nlp = spacy.blank('ru')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data
    }
)

In [ ]:
query = 'Рандомный текст для теста.'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 0.9988330061487115, 'nonetype': 0.001166993851288404}


In [ ]:
%%time
query = 'Размер обеспечения исполнения Контракта составляет 5 (пять) процентов цены контракта'
print(nlp(query)._.cats)


{'обеспечение исполнения контракта': 2.279547603489097e-06, 'nonetype': 0.9999977204523967}
CPU times: user 43.7 ms, sys: 1 ms, total: 44.7 ms
Wall time: 48.5 ms


результат классификаии обратный - чем ближе текст к искомому, тем меньше значение. это связано с плохой работой русскоязычных моделей. classy classification на английском работает отлично 

In [ ]:
df_contract = df[df['label'] == 'обеспечение исполнения контракта'].reset_index(drop=True)
df_contract.shape

(988, 5)

In [ ]:
%%time
predictions = []

for i in range(len(df_contract)):
  result = {}
  for sentence in df_contract.loc[i, 'text']:
    result[sentence] = nlp(sentence)._.cats['обеспечение исполнения контракта']

  predictions.append(choose_minimum_rate_sentence(result)[0])


CPU times: user 30min 52s, sys: 3.61 s, total: 30min 56s
Wall time: 31min 13s


In [ ]:
accuracy_score(predictions, list(df_contract.loc[:, 'target'].values))

0.4898785425101215

Видим, что c русским языком модель работает не очень стабильно, но результат accuracy 0.51 выглядит неплохо. 

Ещё один минус такой модели - долгая работа. Т.к. модель не обучается, а каждый раз считает результат на основе входных данных. 

Построим вторую часть модель для label обеспечение гарантийных обязательств

In [ ]:
garantee_enforcement_100 = np.random.choice(garantee_enforcement, 100, replace=False)
nonetype_100 = np.random.choice(nonetype, 100, replace=False)



data={
    'обеспечение гарантийных обязательств': list(garantee_enforcement_100),
    'nonetype': list(nonetype_100)}
nlp = spacy.blank('ru')
nlp.add_pipe(
    'text_categorizer',
    config={
        'data': data
    }
)

In [ ]:
df_garantee = df[df['label'] == 'обеспечение гарантийных обязательств'].reset_index(drop=True)
df_garantee.shape

(811, 5)

In [ ]:
%%time
predictions = []

for i in range(len(df_garantee)):
  result = {}
  for sentence in df_garantee.loc[i, 'text']:
    result[sentence] = nlp(sentence)._.cats['обеспечение гарантийных обязательств']

  predictions.append(choose_minimum_rate_sentence(result,  min_rate=0.1)[0])

CPU times: user 21min 47s, sys: 2.64 s, total: 21min 50s
Wall time: 21min 57s


In [ ]:
accuracy_score(predictions, list(df_garantee.loc[:, 'target'].values))

0.5154130702836005

### Построим предсказатель для неразмеченных данных

In [ ]:
class ClassyClassificator():
  def __init__(self, contract_enforcement_list, nonetype_list):
    passЕ